## Testing the Components
-----

In [12]:
import tensorflow as tf
assert len(tf.config.list_physical_devices('GPU')) > 0, "No GPU."

In [13]:
from importlib import reload

from src.data.states import get_body_state, get_astrotime_now

import src.components as cpts; cpts = reload(cpts)
import src.plot as cplot; cplot = reload(cplot)

In [14]:
step_size = 2.

time = get_astrotime_now()
walker_name = "mars"
walker_position, walker_velocity = get_body_state(walker_name, time).values()

#### Testing the `Walker` / `SunSystem` / `Solver` Composition
-----

In [15]:
system = cpts.SunSystem(["earth", "jupiter"], step_size=step_size) 
walker = cpts.Walker(
    walker_position, walker_velocity, 
    system, 
    name=f"walker ({walker_name})"
)
solver = cpts.Solver(step_size=step_size)

In [16]:
for _ in range(500):
    walker.propagate(solver)
    system.propagate()

In [17]:
plotter = cplot.Plotter({
    'walker': walker,
    'system': system,
})
fig = plotter.draw(mode="3d")

#### Testing the `Target`s

----

In [18]:
walker_name = "mars"
walker_position, walker_velocity = get_body_state(walker_name, time).values()

In [19]:
system = cpts.SunSystem(["earth", "jupiter"], step_size=step_size)
walker = cpts.Walker(
    walker_position, walker_velocity, 
    system, 
    name=f"walker ({walker_name})"
)
solver = cpts.Solver(step_size=step_size)
target = cpts.OrbitTarget("mars", step_size=step_size, tilt_angle=45)
# target = cpts.FixedTarget([0, 0, 2])

In [20]:
for _ in range(500):
    walker.propagate(solver)
    system.propagate()
    target.propagate()

In [21]:
target.state_vector

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 1.3355612 , -0.27492848,  0.2376497 ,  1.3355612 , -0.31079325,
       -0.19335975], dtype=float32)>

In [22]:
plotter = cplot.Plotter({
    'walker': walker,
    'system': system,
    'target': target
})
fig = plotter.draw(mode="3d")